## 1. Load data from file

In [0]:
import numpy as np

DATA_PATH = './mnist.csv'
SIZE_ROW = 28
SIZE_COL = 28

# load data from file
data = np.genfromtxt(DATA_PATH, delimiter=',')

# separate pixel values from labels
X_raw = data[:, 1:]
Y_raw = data[:, 0]

## 2. Data preprocessing

In [0]:
# normalize X data
X_data = (X_raw - np.min(X_raw)) / (np.max(X_raw) - np.min(X_raw))

# one-hot encoding Y data
Y_data = Y_raw.astype('int')
Y_data = np.eye(np.unique(Y_data).shape[0])[Y_data]

# train data
X_train = X_data[:6000]
Y_train = Y_data[:6000]

# test data
X_test = X_data[6000:]
Y_test = Y_data[6000:]

## 3. Define model and functions for learning neural network

In [0]:
def initialize_weights(fan_in, fan_out):
    return np.sqrt(1 / fan_in) * np.random.randn(fan_out, fan_in)

def activation(z):
  return 1 / (1 + np.exp(-z))

def objective(Y_pred, Y):
  epsilon = 1e-8
  return (-1 / Y.shape[0]) * np.sum(
      Y * np.log(Y_pred + epsilon) + (1 - Y) * np.log(1 - Y_pred + epsilon)
  )

def accuracy(Y_pred, Y):
  answer_Y_pred = np.argmax(Y_pred, axis=1)
  answer_Y = np.argmax(Y, axis=1)
  return np.mean(answer_Y_pred == answer_Y)


class ThreeLayerNN:
  def __init__(self, input_size, hidden1_size, hidden2_size, output_size):
    self.params = dict()
    self.forwards = dict()
    self.grads = dict()

    self.params['w1'] = initialize_weights(input_size, hidden1_size)
    self.params['b1'] = np.zeros((1, hidden1_size))

    self.params['w2'] = initialize_weights(hidden1_size, hidden2_size)
    self.params['b2'] = np.zeros((1, hidden2_size))

    self.params['w3'] = initialize_weights(hidden2_size, output_size)
    self.params['b3'] = np.zeors((1, output_size))
    
  def forward(self, X):
    w1, w2, w3 = self.params['w1'], self.params['w2'], self.params['w3']
    b1, b2, b3 = self.params['b1'], self.params['b2'], self.params['b3']

    forward_results = dict()

    forward_results['z1'] = np.matmul(X, w1.T) + b1
    forward_results['a1'] = activation(z1)

    forward_results['z2'] = np.matmul(a1, w2.T) + b2
    forward_results['a2'] = activation(z2)

    forward_results['z3'] = np.matmul(a2, w3.T) + b3
    forward_results['a3'] = activation(z3)

    return forward_results

  def backward(self, X, Y, forward_results):
    w1, w2, w3 = self.params['w1'], self.params['w2'], self.params['w3']
    b1, b2, b3 = self.params['b1'], self.params['b2'], self.params['b3']

    z1, z2, z3 = forward_results['z1'], forward_results['z2'], forward_results['z3']
    a1, a2, a3 = forward_results['a1'], forward_results['a2'], forward_results['a3']

    dz3 = (a3 - Y) / X.shape[0]

    da2 = np.matmul(dz3, w3)
    dz2 = a2 * (1 - a2) * da2

    da1 = np.matmul(dz2, w2)
    dz1 = a1 * (1 - a1) * da1

    self.grads['w3'] = np.matmul(dz3.T, a2)
    self.grads['b3'] = np.sum(dz3, axis=0).reshape(1, -1)

    self.grads['w2'] = np.matmul(dz2.T, a1)
    self.grads['b2'] = np.sum(dz2, axis=0).reshape(1, -1)

    self.grads['w1'] = np.matmul(dz1.T, X)
    self.grads['b1'] = np.sum(dz1, axis=0).reshape(1, -1)

## 4. Learning with the gradient descent algorithm


In [0]:
lr = 0.5
epoch_count = 20000
nn = ThreeLayerNN(784, 196, 49, 10)

history = {
    'train_loss': [],
    'test_loss': [],
    'train_acc': [],
    'test_acc': []
}

for epoch in range(epoch_count):
  # forward propagation using train data
  train_forward_results = nn.forward(X_train)

  # calculate training loss and accuracy
  Y_train_pred = train_forward_results['a3']
  train_loss = objective(Y_train_pred, Y_train)
  train_acc = accuracy(Y_train_pred, Y_train)

  # forward propagation using test data
  test_forward_results = nn.forward(X_test)

  # calculate testing loss and accuracy
  Y_test_pred = test_forward_results['a3']
  test_loss = objective(Y_test_pred, Y_test)
  test_acc = accuracy(Y_test_pred, Y_test)

  # log history
  history['train_loss'].append(train_loss)
  history['test_loss'].append(test_loss)
  history['train_acc'].append(train_acc)
  history['test_acc'].append(test_acc)

  # gradient descent
  nn.backward(X_train, Y_train, train_forward_results)
  for key in nn.params.keys():
    nn.params[key] -= lr * nn.grads[key]

# convert list to numpy array
for key in history.keys():
  history[key] = np.array(history[key])